# <center> MATH&ML-14. Рекомендательные системы. Часть I. Практика

In [158]:
#Импорты
import pandas as pd
import numpy as np
import math

In [159]:
sa = pd.read_csv('data/shared_articles.zip')

display(sa.head(3))
display(sa.info())
display(sa.shape)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3122 entries, 0 to 3121
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        3122 non-null   int64 
 1   eventType        3122 non-null   object
 2   contentId        3122 non-null   int64 
 3   authorPersonId   3122 non-null   int64 
 4   authorSessionId  3122 non-null   int64 
 5   authorUserAgent  680 non-null    object
 6   authorRegion     680 non-null    object
 7   authorCountry    680 non-null    object
 8   contentType      3122 non-null   object
 9   url              3122 non-null   object
 10  title            3122 non-null   object
 11  text             3122 non-null   object
 12  lang             3122 non-null   object
dtypes: int64(4), object(9)
memory usage: 317.2+ KB


None

(3122, 13)

Информация в столбцах:  
дата публикации (временная метка),   
исходный URL-адрес,  
заголовок,  
содержание в виде обычного текста,  
язык статьи (португальский — pt или английский — en),  
информация о пользователе, который поделился статьёй (автор).

CONTENT SHARED — статья была опубликована на платформе и доступна для пользователей;  
CONTENT REMOVED — статья была удалена с платформы и недоступна для дальнейших рекомендаций.

In [160]:
#Задание 6.1
sa_content_shared_filtered = sa[sa['eventType']=='CONTENT SHARED']
sa_content_shared_filtered.shape

(3047, 13)

In [161]:
ui = pd.read_csv('data/users_interactions.zip')

display(ui.head(3))
display(ui.info())
display(ui.shape)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72312 entries, 0 to 72311
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    72312 non-null  int64 
 1   eventType    72312 non-null  object
 2   contentId    72312 non-null  int64 
 3   personId     72312 non-null  int64 
 4   sessionId    72312 non-null  int64 
 5   userAgent    56918 non-null  object
 6   userRegion   56907 non-null  object
 7   userCountry  56918 non-null  object
dtypes: int64(4), object(4)
memory usage: 4.4+ MB


None

(72312, 8)

В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:

VIEW — просмотр,  
LIKE — лайк,  
COMMENT CREATED — комментарий,  
FOLLOW — подписка,  
BOOKMARK — добавление в закладки.  

In [162]:
#Преобразования из урока
ui.personId = ui.personId.astype(str)
ui.contentId = ui.contentId.astype(str)
sa_content_shared_filtered.contentId = sa_content_shared_filtered.contentId.astype(str)

C:\Users\telis\AppData\Local\Temp\ipykernel_5580\879140087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_content_shared_filtered.contentId = sa_content_shared_filtered.contentId.astype(str)


In [163]:
#Придадим вес пользовательской активности
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [164]:
#Задание 6.2
ui['event_weight'] = ui['eventType'].apply(lambda x: event_type[x])
display(ui)

print('Среднее значение весов в данных:', round(ui['event_weight'].mean(), 2))

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,event_weight
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


Среднее значение весов в данных: 1.24


In [165]:
#Задание 6.3

users_interactions_count_df = (
    ui.groupby(['personId', 'contentId']).first()
    .reset_index().groupby('personId').size()
    )

users_with_5_or_more_iteractions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

print('Количество пользователей, взаимодействующих хотя бы с 5 статьями:', 
      len(users_with_5_or_more_iteractions_df))

Количество пользователей, взаимодействующих хотя бы с 5 статьями: 1140


In [166]:
#Задание 6.4

filtered_ui = ui[ui['personId'].isin(users_with_5_or_more_iteractions_df['personId'].tolist())]
print('Действий пользовтелей, у которых минимум 5 взаимодействий:', filtered_ui.shape[0])

Действий пользовтелей, у которых минимум 5 взаимодействий: 69868


In [167]:
# #Задание 6.5

# #Этот вариант как по мне более корректный судя по заданию, 
# # но с эталонным решением и ответом не совпадает

# #Функция для логарифмирования весов действий пользователей
# def smooth_user_preference(x):
#     return math.log(1+x, 2)

# filtered_content_by_user = filtered_ui.groupby('personId')['event_weight'].sum().reset_index()
# filtered_content_by_user_time = pd.merge(filtered_content_by_user, filtered_ui.groupby(['personId'])['timestamp'].max().reset_index(), on='personId', how='inner')
# filtered_content_by_user_time['event_weight_log'] = filtered_content_by_user_time['event_weight'].apply(smooth_user_preference)
# display(filtered_content_by_user_time)

# print('Среднее по признаку с последними временными отсечками:', round(filtered_content_by_user_time['timestamp'].mean(), 2))

In [168]:
#Задание 6.5

#Функция для логарифмирования весов действий пользователей
def smooth_user_preference(x):
    return math.log(1+x, 2)

filtered_content_by_user = filtered_ui.groupby('personId')['event_weight'].sum().reset_index()
filtered_content_by_user_time = pd.merge(filtered_content_by_user, filtered_ui.groupby(['personId', 'contentId'])['timestamp'].max().reset_index(), on='personId', how='left')
filtered_content_by_user_time['event_weight_log'] = filtered_content_by_user_time['event_weight'].apply(smooth_user_preference)
display(filtered_content_by_user_time)

print('Среднее по признаку с последними временными отсечками:', round(filtered_content_by_user_time['timestamp'].mean(), 2))

,personId,event_weight,contentId,timestamp,event_weight_log
0,-1007001694607905623,7.0,-5065077552540450930,1470395911,3.000000
1,-1007001694607905623,7.0,-6623581327558800021,1487240080,3.000000
2,-1007001694607905623,7.0,-793729620925729327,1472834892,3.000000
3,-1007001694607905623,7.0,1469580151036142903,1487240062,3.000000
4,-1007001694607905623,7.0,7270966256391553686,1485994342,3.000000
...,...,...,...,...,...
39101,998688566268269815,13.0,-401664538366009049,1474567449,3.807355
39102,998688566268269815,13.0,3456674717452933449,1478802088,3.807355
39103,998688566268269815,13.0,6881796783400625893,1474567675,3.807355
39104,998688566268269815,13.0,7174452660053929140,1478812905,3.807355


Среднее по признаку с последними временными отсечками: 1470605340.04


In [169]:
#Задание 6.6
train_df = filtered_content_by_user_time[filtered_content_by_user_time['timestamp'] < 1475519545]
test_df = filtered_content_by_user_time[filtered_content_by_user_time['timestamp'] > 1475519545]

train_df

,personId,event_weight,contentId,timestamp,event_weight_log
0,-1007001694607905623,7.0,-5065077552540450930,1470395911,3.000000
2,-1007001694607905623,7.0,-793729620925729327,1472834892,3.000000
6,-1032019229384696495,2943.5,-1006791494035379303,1469129122,11.523807
7,-1032019229384696495,2943.5,-1039912738963181810,1459376415,11.523807
8,-1032019229384696495,2943.5,-1081723567492738167,1464054096,11.523807
...,...,...,...,...,...
39099,997469202936578234,42.5,9112765177685685246,1472479493,5.442943
39100,998688566268269815,13.0,-1255189867397298842,1474567164,3.807355
39101,998688566268269815,13.0,-401664538366009049,1474567449,3.807355
39103,998688566268269815,13.0,6881796783400625893,1474567675,3.807355


In [170]:
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


In [171]:
#Задание 6.7
popular_content = train_df.groupby('contentId')['event_weight_log'].agg('sum').reset_index().sort_values('event_weight_log', ascending=False)
popular_content

,contentId,event_weight_log
43,-133139342397538859,1029.544533
805,-6783772548752091658,946.572471
2111,7507067965574797372,842.218273
1002,-8208801367848627943,839.373042
2211,8224860111193157980,752.679472
...,...,...
1897,5937899505996968869,3.807355
1768,5062670850994683718,3.700440
2301,8872925201254666834,3.700440
1982,6569723785655740276,2.807355


In [175]:
#Эталон
#Тоже не сошелся
popular = (
    train_df
    .groupby('contentId')
    .event_weight_log.sum().reset_index()
    .sort_values('event_weight_log', ascending=False)
    .contentId.values
)
popular

array(['-133139342397538859', '-6783772548752091658',
       '7507067965574797372', ..., '8872925201254666834',
       '6569723785655740276', '-3113913063173722290'], dtype=object)

In [173]:
#Задание 6.8

In [ ]:
#Эталон
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
calc_precision('popular')

0.0058746934408905665